In [ ]:
#Uncomment this part if not already installed

'''
!apt update
!apt install -y firefox
!wget -q https://github.com/mozilla/geckodriver/releases/latest/download/geckodriver-linux64.tar.gz
!tar -xvzf geckodriver-linux64.tar.gz
!chmod +x geckodriver
!mv geckodriver /usr/local/bin/
'''

'\n!apt update\n!apt install -y firefox\n!wget -q https://github.com/mozilla/geckodriver/releases/latest/download/geckodriver-linux64.tar.gz\n!tar -xvzf geckodriver-linux64.tar.gz\n!chmod +x geckodriver\n!mv geckodriver /usr/local/bin/\n'

In [ ]:
#Uncomment this part if not already installed
'''
!apt update > /dev/null
!apt install -y firefox-esr > /dev/null
!wget https://github.com/mozilla/geckodriver/releases/download/v0.33.0/geckodriver-v0.33.0-linux64.tar.gz > /dev/null
!tar -xvzf geckodriver-v0.33.0-linux64.tar.gz > /dev/null
!mv geckodriver /usr/local/bin/
!pip install selenium > /dev/null
'''


**LOG IN TO TILEDB**

In [ ]:
from getpass import getpass

username = input("Enter your TileDB username (email): ")
password = getpass("Enter your TileDB password: ")


In [ ]:
import os, time, traceback
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def create_logged_in_session(username, password):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Firefox(options=options)
    driver.get("https://www.particle.cz/work/TileDB2/account/login")

    try:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "MainContent_loginEmailTxt")))
        driver.find_element(By.ID, "MainContent_loginEmailTxt").send_keys(username)
        driver.find_element(By.ID, "MainContent_loginPasswordTxt").send_keys(password)
        driver.find_element(By.ID, "MainContent_LoginBtn").click()
        WebDriverWait(driver, 15).until(EC.url_contains("TileDB2"))
        print("[✅] Logged in successfully.")
        return driver
    except Exception as e:
        print("[❌] Login failed.")
        driver.save_screenshot("login_error.png")
        with open("login_error.html", "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        raise e


In [ ]:
def batch_register(cli, base_path="Burn_In"):
    from datetime import datetime

    os.makedirs("screenshots", exist_ok=True)
    os.makedirs("html_dumps", exist_ok=True)

    for file_name in sorted(os.listdir(base_path)):
        if not file_name.lower().endswith(".txt"):
            continue

        try:
            # Extract Brick ID (digits only) from filename like SA-076_6.TXT
            raw_id = ''.join(filter(str.isdigit, file_name.split('_')[0]))
            if len(raw_id) < 3:
                print(f"[⚠️] Skipping '{file_name}' – couldn't parse brick ID.")
                continue
            brick_id = f"FSA{raw_id}".upper()[:6]

            # Fallback: use today's date
            date_obj = datetime.today()

            print(f"\nRegistering: Brick '{brick_id}' from '{file_name}'")

            # Manufacture form
            cli.get("https://www.particle.cz/work/TileDB2/brick10v/manufacture")
            WebDriverWait(cli, 15).until(EC.presence_of_element_located((By.ID, "MainContent_brickCodeTxt")))

            # Uncheck prefix
            try:
                prefix_checkbox = cli.find_element(By.ID, "MainContent_brickCodePrefixEnabled")
                if prefix_checkbox.is_selected():
                    prefix_checkbox.click()
                    print("[🧹] Unchecked 'Include code prefix'")
            except:
                pass

            # Brick ID
            cli.find_element(By.ID, "MainContent_brickCodeTxt").send_keys(brick_id)

            # Location
            location_input = cli.find_element(By.ID, "MainContent_brickLocationCB_brickLocationCB_TextBox")
            location_input.clear()
            location_input.send_keys("Wits")
            time.sleep(1)
            location_input.send_keys(Keys.TAB)

            # Manufacture Date
            formatted_date = date_obj.strftime("%d.%m.%Y 00:00 AM")
            date_input = cli.find_element(By.ID, "MainContent_brickCreatedTxt")
            date_input.clear()
            date_input.send_keys(formatted_date)
            date_input.send_keys(Keys.TAB)

            # Note
            note_input = cli.find_element(By.ID, "MainContent_brickNoteTxt")
            note_input.clear()
            note_input.send_keys("Completed Burn-in Testing")

            # Save
            WebDriverWait(cli, 10).until(EC.element_to_be_clickable((By.ID, "MainContent_brickSaveBtn"))).click()
            time.sleep(2)

            try:
                WebDriverWait(cli, 3).until(EC.alert_is_present())
                alert = cli.switch_to.alert
                print(f"[⚠️] Alert: {alert.text}")
                alert.accept()
                continue  # Skip measurement if already exists
            except:
                print(f"[✅] Brick '{brick_id}' registered.")

            # Upload .txt file
            upload_diary_record(cli, brick_id, os.path.join(base_path, file_name))

        except Exception as e:
            print(f"[❌] Error with '{file_name}': {e}")
            cli.save_screenshot(f"screenshots/error_{brick_id}.png")
            with open(f"html_dumps/error_{brick_id}.html", "w", encoding="utf-8") as f:
                f.write(cli.page_source)
            traceback.print_exc()


In [ ]:
def upload_diary_record(cli, brick_id, filepath):
    from datetime import datetime

    print(f"📎 Attaching diary file for '{brick_id}'")

    cli.get(f"https://www.particle.cz/work/TileDB2/brick10v/measure?partcode={brick_id}")
    WebDriverWait(cli, 15).until(EC.presence_of_element_located((By.ID, "MainContent_measureFileUpl")))

    # File input
    file_input = cli.find_element(By.ID, "MainContent_measureFileUpl")
    file_input.send_keys(os.path.abspath(filepath))

    # Author
    author_box = cli.find_element(By.ID, "MainContent_measureAuthorCB_measureAuthorCB_TextBox")
    author_box.clear()
    author_box.send_keys("Thabo Pilusa")
    time.sleep(1)
    author_box.send_keys(Keys.TAB)

    # Date
    now_str = datetime.now().strftime("%d.%m.%Y %H:%M")
    measure_date_input = cli.find_element(By.ID, "MainContent_brickMeasuredTxt")
    measure_date_input.clear()
    measure_date_input.send_keys(now_str)

    # Note
    note_box = cli.find_element(By.ID, "MainContent_measureNoteTxt")
    note_box.clear()
    note_box.send_keys("Final test upload")

    # Submit
    WebDriverWait(cli, 10).until(EC.element_to_be_clickable((By.ID, "MainContent_brickMeasureBtn"))).click()
    print(f"[✅] Uploaded diary file for '{brick_id}'")


In [ ]:
cli = create_logged_in_session(username, password)
batch_register(cli, base_path="burnin_dataset_folder")
cli.quit()
